In [1]:
from sklearn import preprocessing
#Set Seed
import random
import os 
import pandas as pd 
import numpy as np
import torch
random.seed(555)
np.random.seed(555)
torch.manual_seed(555)

In [2]:
# Install packages

#!unzip millestone_3.zip

!pip install tf-estimator-nightly==2.8.0.dev2021122109

!pip install pytorch_widedeep

!pip install munkres

In [4]:
from sklearn.manifold import TSNE

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from statistics import mean, median, variancefrom pytorch_widedeep import Trainer, Tab2Vec
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, WideDeep, TabNet, TabTransformer
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep.datasets import load_adult
import random
random.seed(555) #
np.random.seed(555)
torch.manual_seed(555)



file_n = pd.read_csv('TextPre.csv', sep=',', on_bad_lines='skip')
# ......Choose table embedding model
# TabTransformer, TabNet (output dim TabTransformer = 208, TabNet= 693)
table_emb_model = 'TabNet'

vectors = []
labels = []
texts = []
dim_size = []

error = 0
for i in range(len(file_n['file_names'])):
    df_ = pd.read_csv('Data/Tables/'+file_n['file_names'][i]+'.csv', sep=',', on_bad_lines='skip')
    df_ = df_.replace('����|NULL|Null|null|&NBSP|&Nbsp|nbsp|nbsp;|????|�|Nbsp|nbsp','')
    df_ = df_.replace('&nbsp;','',regex=True)
    df_ = df_.replace('NaN','',regex=True)
    
  
    # Identify category and continious data
    cols = df_.columns
    cont_cols = df_._get_numeric_data().columns 
    cat_cols = list(set(cols) - set(cont_cols))
    
        
    # Fit the TabPreprocessor
    tab_preprocessor = TabPreprocessor(cat_embed_cols=cat_cols, continuous_cols=cont_cols,scale=False)  
    
    
    try:
        X_tab = tab_preprocessor.fit_transform(df_)
    except:
        error = error +1
    else:
        
        # Define the model (and let's assume we train it)
        
        if table_emb_model == 'TabTransformer':
            md = TabFastFormer
        if table_emb_model == 'TabNet':
            md = TabNet    
       
        
        tabmodel = md(column_idx=tab_preprocessor.column_idx,
                        cat_embed_input=tab_preprocessor.cat_embed_input,
                        continuous_cols=tab_preprocessor.continuous_cols) #TabTransformer input_dim = 16
        
       
        model = WideDeep(deeptabular=tabmodel, pred_dim =384)  
        
        # Emb in 128 dim 
        t2v = Tab2Vec(model, tab_preprocessor)
        try:
            X_vec = t2v.transform(df_) # df_
        except:
            error = error + 1
        else:
            m_size=X_vec.shape
            print(X_vec.shape)
            print(m_size[1])
            dim_size.append(m_size[1])
           # Compute mean to embedding and vector to list vectors
            vectors.append(np.mean(X_vec ,axis=0))
            labels.append(file_n['Labels'][i])
            texts.append(file_n['Text'][i])   

In [39]:
import statistics
dim_size=max(dim_size)
print('Maximum time occurred dim size = ' + str(dim_size))
#vectors
dim = dim_size   # for TabTransformer dim_size-1,for TabNet only dim_size and dim TabTransformer = 208, TabNet= 693
df = pd.DataFrame(vectors)
df= df.iloc[: , :dim_size]
df=df.interpolate(method='linear', limit_direction = 'both')
vectors = df.to_numpy()

Maximum time occurred dim size = 693


In [40]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
def sdcn_preprocessing(x,y):
    with open('X.txt', 'w', encoding='utf8') as data:
        x_str = ''
        for i, xx in enumerate(x):
            x_str = x_str + str(xx[0])
            for j in range(1, len(xx)-1):
                x_str = x_str + ' ' + str(xx[j])
            x_str = x_str + ' ' + str(xx[len(xx)-1]) + '\n'
        data.write(x_str)
    with open('labels.txt', 'w', encoding='utf8') as label:
        y_str = ''
        for yy in y :
            y_str = y_str + str(yy) + '\n'
        label.write(y_str)

sdcn_preprocessing(vectors,y)
